In [ ]:
import os
os.getcwd()

In [ ]:
!pip install --upgrade pip
!pip install pyqt5-tools
!pip install PyQt5
!pip install opencv-python
!pip install --upgrade pyqt5 lxml
!pip install wget
!pip install numpy
!pip install scipy
!pip install apache-beam
!pip install avro-python3
!pip install contextlib2
!pip install Cython
!pip install lvis
!pip install matplotlib
!pip install pandas
!pip install Pillow
!pip install pycocotools
!pip install gin-config
!pip install google-api-python-client
!pip install kaggle
!pip install oauth2client
!pip install opencv-python-headless
!pip install py-cpuinfo
!pip install PyYAML
!pip install sacrebleu
!pip install sentencepiece
!pip install seqeval
!pip install pyparsing

##### Don't install #####
!pip install tensorflow
!pip install tensorflow-addons
!pip install tensorflow-datasets
!pip install tensorflow-hub
!pip install tensorflow-model-optimization
!pip install tensorflow-text

In [ ]:
import cv2
import os
import wget
import numpy
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
labels = ['base', 'end']
number_imgs = 2
CUSTOM_MODEL_NAME = 'my_stick_ssd_mobnet2' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'stick_label_map2.pbtxt'

In [3]:
paths = {
    'SCRIPTS_PATH': os.path.join('tf', 'scripts'),
    'APIMODEL_PATH': os.path.join('tf', 'Model'),
    'ANNOTATION_PATH': os.path.join('annotations'),
    'IMAGE_PATH': '',
    'PRETRAINED_MODEL_PATH': os.path.join('pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Model', CUSTOM_MODEL_NAME), 
    'PROTOC_PATH':os.path.join('protoc')
 }

files = {
    'PIPELINE_CONFIG':os.path.join(paths['CHECKPOINT_PATH'], 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
wget.download(url)
!move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}

In [ ]:
!cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
!cd tf/Model/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
!cd tf/Model/research/slim && pip install -e . 

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')


In [ ]:
!python {VERIFICATION_SCRIPT}

In [1]:
import object_detection

In [3]:
print(os.getcwd())

c:\Users\thana\Desktop\TheShit\DurianSang\Mobnet


In [2]:
!pip show object_detection

Name: object-detection
Version: 0.1
Summary: Tensorflow Object Detection Library
Home-page: UNKNOWN
Author: 
Author-email: 
License: UNKNOWN
Location: c:\users\thana\desktop\theshit\duriansang\mobnet\lib\site-packages\object_detection-0.1-py3.7.egg
Requires: apache-beam, avro-python3, contextlib2, Cython, keras, lvis, lxml, matplotlib, pandas, pillow, pycocotools, pyparsing, scipy, six, tensorflow_io, tf-models-official, tf-slim
Required-by: 


In [ ]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

In [ ]:
labels = [{'name':'base', 'id':1}, {'name':'end', 'id':2}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'Train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'Train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'Val')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'Val.record')} 

In [ ]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
print(config)

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'Train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'Val.record')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)   

In [4]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
print(config)

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=200000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [ ]:
print(command)

In [ ]:
!{command}

In [5]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])
print(command)

python tf\Model\research\object_detection\model_main_tf2.py --model_dir=Model\my_stick_ssd_mobnet2 --pipeline_config_path=Model\my_stick_ssd_mobnet2\pipeline.config --checkpoint_dir=Model\my_stick_ssd_mobnet2


In [ ]:
!{command}